In [1]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,613860,32.8,1330087,71.1,1124983,60.1
Vcells,1149311,8.8,8388608,64.0,1801055,13.8


Loading required package: data.table



In [2]:
#Parametros del script
#setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory
setwd("~/buckets/b1/")

#cargo el dataset
dataset  <- fread("./datasets/alt_c2_reparacNA.csv.gz")   #donde entreno# FIN Parametros del script

In [3]:

AgregarVariables  <- function( dataset )
{
  gc()
  #INICIO de la seccion donde se deben hacer cambios con variables nuevas

  #creo un ctr_quarter que tenga en cuenta cuando los clientes hace 3 menos meses que estan
  dataset[  , ctrx_quarter_normalizado := ctrx_quarter ]
  dataset[ cliente_antiguedad==1 , ctrx_quarter_normalizado := ctrx_quarter * 5 ]
  dataset[ cliente_antiguedad==2 , ctrx_quarter_normalizado := ctrx_quarter * 2 ]
  dataset[ cliente_antiguedad==3 , ctrx_quarter_normalizado := ctrx_quarter * 1.2 ]
  
  #variable extraida de una tesis de maestria de Irlanda
  dataset[  , mpayroll_sobre_edad  := mpayroll / cliente_edad ]
  dataset[ , vm_msaldopesos          := rowSums( cbind( Master_msaldopesos,  Visa_msaldopesos) , na.rm=TRUE ) ]
  dataset[ , vm_Fvencimiento         := pmin( Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE) ]
  dataset[ , vm_fechaalta            := pmax( Master_fechaalta, Visa_fechaalta, na.rm = TRUE) ]
  dataset[ , vmr_msaldototal         := vm_msaldototal / vm_mlimitecompra ]
  dataset[ , vmr_mpagominimo         := vm_mpagominimo  / vm_mlimitecompra ]

    #MIAS
    dataset[ , mprestamos                 := rowSums( cbind( mprestamos_personales, mprestamos_prendarios, mprestamos_hipotecarios) , na.rm=TRUE ) ]
    dataset[ , mdebitos_automaticos                := rowSums( cbind( mcuenta_debitos_automaticos,mttarjeta_visa_debitos_automaticos,mttarjeta_master_debitos_automaticos ) , na.rm=TRUE ) ]
    dataset[ , mpago_servicios_total                := rowSums( cbind( mpagodeservicios,mpagomiscuentas ) , na.rm=TRUE ) ]
    dataset[ , mtarjeta_credito_consumos    := rowSums( cbind(mtarjeta_visa_consumo,mtarjeta_master_consumo ) , na.rm=TRUE ) ]
    dataset[ , cpayroll_total    := rowSums( cbind(cpayroll_trx,cpayroll2_trx ) , na.rm=TRUE ) ]
   dataset[ , mpayroll_total    := rowSums( cbind(mpayroll,mpayroll2 ) , na.rm=TRUE ) ]
    dataset[ , minversion1                := rowSums( cbind( minversion1_pesos, minversion1_dolares) , na.rm=TRUE ) ]
    dataset[ , minversion12                := rowSums( cbind( minversion1, minversion2) , na.rm=TRUE ) ]
    dataset[ , minversion_total                := rowSums( cbind( minversion12, mplazo_fijo) , na.rm=TRUE ) ]

    dataset[ , mgasto_mensual_estim    := rowSums( cbind(mdebitos_automaticos,mpago_servicios_total,mtransferencias_emitidas,mextraccion_autoservicio,mcheques_emitidos,ccajas_extracciones,mautoservicio,mtarjeta_credito_consumos ) , na.rm=TRUE ) ]
    dataset[ , mingreso_mensual_estim    := rowSums( cbind(mtransferencias_recibidas,mcheques_depositados,ccajas_depositos,mpayroll_total ) , na.rm=TRUE ) ]
    dataset[ , mcuentas_saldo_mas_invers    := rowSums( cbind(mcuentas_saldo,minversion_total ) , na.rm=TRUE ) ]

    # Elimino las intermedias
    dataset[ , mdebitos_automaticos  := NULL ]
    dataset[ , mpago_servicios_total := NULL ]
    dataset[ ,  minversion12         := NULL ]
    dataset[ , minversion1           := NULL ]
    dataset[ , minversion_total      := NULL ]

  
  #valvula de seguridad para evitar valores infinitos
  #paso los infinitos a NULOS
  infinitos      <- lapply(names(dataset),function(.name) dataset[ , sum(is.infinite(get(.name)))])
  infinitos_qty  <- sum( unlist( infinitos) )
  if( infinitos_qty > 0 )
  {
    cat( "ATENCION, hay", infinitos_qty, "valores infinitos en tu dataset. Seran pasados a NA\n" )
    dataset[mapply(is.infinite, dataset)] <<- NA
  }


  #valvula de seguridad para evitar valores NaN  que es 0/0
  #paso los NaN a 0 , decision polemica si las hay
  #se invita a asignar un valor razonable segun la semantica del campo creado
  nans      <- lapply(names(dataset),function(.name) dataset[ , sum(is.nan(get(.name)))])
  nans_qty  <- sum( unlist( nans) )
  if( nans_qty > 0 )
  {
    cat( "ATENCION, hay", nans_qty, "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n" )
    cat( "Si no te gusta la decision, modifica a gusto el programa!\n\n")
    dataset[mapply(is.nan, dataset)] <<- 0
  }

}

In [4]:
#primero agrego las variables manuales
AgregarVariables( dataset )

Warning message in `[.data.table`(dataset, cliente_antiguedad == 3, `:=`(ctrx_quarter_normalizado, :
“2.400000 (type 'double') at RHS position 1 truncated (precision lost) when assigning to type 'integer' (column 154 named 'ctrx_quarter_normalizado')”


ERROR: Error in ifelse(mcuentas_saldo == 0, 0, mv_mlimitecompra/mcuentas_saldo): object 'mv_mlimitecompra' not found


In [ ]:
fwrite( dataset,
        file="./datasets/alt_c2_reparacNA_feinicialymio.csv.gz",
        sep= "," )

In [ ]:
nrow(dataset)

In [ ]:
ncol(dataset)